In [1]:
#Initial imports 
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener 
from tweepy import OAuthHandler #HandlerAuthorization
from tweepy import Stream


#Importing pandas and numpy 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Importing texblob which is a library that help us to doing Sentiment Analysis by cleanin
from textblob import TextBlob
import re

import json
import tweepy

In [2]:
#Creation of user credetials
#Install pip install tweepy

ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""
API_KEY = ""
API_KEY_SECRET = ""
Bearer_Token = "AAAAAAAAAAAAAAAAAAAAAFzETgEAAAAAiFEDdjwmtJP09TG6LvFeTmbfxXo%3D6Mdynrv0VPT32p96YxrdRApnbM6JhPOpwde2kXGty1bggQF2S"

In [ ]:
class ActivandoConexion:
    def __init__(self, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, API_KEY, API_KEY_SECRET):
        self.ACCESS_TOKEN = ACCESS_TOKEN
        self.ACCESS_TOKEN_SECRET = ACCESS_TOKEN_SECRET
        self.API_KEY = API_KEY
        self.API_KEY_SECRET = API_KEY_SECRET
        
    def GenerarAuthorization(self):
        auth = tweepy.OAuthHandler(self.API_KEY,self.API_KEY_SECRET)
        auth.set_access_token(self.ACCESS_TOKEN,self.ACCESS_TOKEN_SECRET)
        return auth
        
    def ConectandoAPI():
        pass
        

class TweetsListener(tweepy.StreamListener):
    
    def on_connect(self):
        print("Online")
        
    def on_status(self, status):
        tweet_to_clean = [status.created_at,              #Pos0
                          status.text,                    #Pos1
                          status.user.followers_count,    #Pos2
                          status.user.friends_count,      #Pos3
                          status.reply_count,             #Pos4
                          status.retweet_count,           #Pos5
                          status.geo,                     #Pos6
                          status.place,                   #Pos7
                          status.user.location,           #Pos8
                          status.user.name]               #Pos9
        activando = ValidarContenido(tweet_to_clean)
        activando.analizarTexto()
                                
    def on_error(self, status_code):
        print("Error", status_code)
        
class ValidarContenido():
    listado_tweets_interes = []
    listado_temporal = []
    lista_especial = ["#SOSGEB", "SOSGEB", "torres", "energia", "falla", "fallas", "transmision", 
                      "energía", "avalancha", "transmisión", "atentado", "ironhack", "Ironhack", "torre"]
    pond_final = 0
    contador = 0
    def __init__(self,nuevo_tweet):
        self.nuevo_tweet = nuevo_tweet
        
    def analizarTexto(self):                 #Validar que realmente vengan dentro del texto las keywords definidas y validar numero de match
        
        self.listado_temporal = []
        print('')
        print('Ha llegado un nuevo Tweet, iniciando analisis...')
        print('==================================================================================================')
        print('El Tweet que llego es:', self.nuevo_tweet[1])
        print('==================================================================================================')
        print('La descomposicion del tweet es:')
        print('Fecha_Creacion',self.nuevo_tweet[0])
        print('Tweet ya impreso',)
        print('followers_count',self.nuevo_tweet[2])
        print('friends_count',self.nuevo_tweet[3])
        print('reply_count',self.nuevo_tweet[4])
        print('retweet_count',self.nuevo_tweet[5])
        print('geo',self.nuevo_tweet[6])
        print('place',self.nuevo_tweet[7])
        print('location',self.nuevo_tweet[8])
        print('name',self.nuevo_tweet[9])
        print('==================================================================================================')
        nombreLista = self.nuevo_tweet[1].split()
        for i in nombreLista:
            if i in self.lista_especial:
                self.contador += 1
        if self.contador >0:
            self.pond_final += 10
            self.listado_temporal.append(self.nuevo_tweet[1]) #Incluyendo el Texto del Tweet
            self.listado_temporal.append(self.nuevo_tweet[9]) #Inlcuyendo nombre de usuario
            self.listado_temporal.append(self.contador)    #Inlcuyendo contador de match                           
            self.analizarUbicacion()
        #si contador = 0 el tweet se desecha  
        else:
            print('==================================================================================================')
            print('El Tweet que llego se desecho por no cumplir con los parametros')
            print('==================================================================================================')
            self.contador = 0
            
    def analizarUbicacion(self):             #Validar ubicacion del tweet y dar mayor ponderacion aquellos que tienen una ubicacion y despues validar si es cerca al lugar de operacion. de interes en colombia   
        if (self.nuevo_tweet[6] !=None):
            self.pond_final +=15
            self.listado_temporal.append(self.nuevo_tweet[6]) #Inlcuyendo ubicacion del usuario
            self.analisisSentimiento()
            
        elif (self.nuevo_tweet[7] !=None):
            self.pond_final +=15
            self.listado_temporal.append(self.nuevo_tweet[7]) #Inlcuyendo ubicacion del usuario
            self.analisisSentimiento()
            
        elif (self.nuevo_tweet[8] !=None):
            self.pond_final +=15
            self.listado_temporal.append(self.nuevo_tweet[8]) #Inlcuyendo ubicacion del usuario
            self.analisisSentimiento()
        
        else:
            self.listado_temporal.append('Sin Ubicacion')
            self.analisisSentimiento()
        
    def clean_tweet(self, tweet): #Cleanin the tweet by regular expression
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
    def analisisSentimiento(self):         
        analysis = TextBlob('"' + self.clean_tweet(self.nuevo_tweet[1] + '"'))
        if analysis.sentiment.polarity > 0:  #This library measures this like a positive tweet
            self.listado_temporal.append(analysis.sentiment.polarity)
        elif analysis.sentiment.polarity == 0:  #This library measures this is uncertain like a positive or negative tweet neutra
            self.listado_temporal.append(analysis.sentiment.polarity)
        else:
            self.listado_temporal.append(analysis.sentiment.polarity) #This library measures this like negative tweet
        self.analisisSeguidores()
    
    def analisisSeguidores(self):             #Analisis de si la persona que escribio el tweet puede Mediatica, Tiene un potencial de viralizacion del tweet mucho mayor
        if (self.nuevo_tweet[2] <= 50):
            self.pond_final +=0
            self.listado_temporal.append(self.nuevo_tweet[2])  #Agregando cantidad de seguidores
            self.analisisAmigos()
        elif (self.nuevo_tweet[2] < 50 and self.nuevo_tweet[2] <= 200  ):
            self.pond_final +=10
            self.listado_temporal.append(self.nuevo_tweet[2])  #Agregando cantidad de seguidores
            self.analisisAmigos()
        elif (self.nuevo_tweet[2] < 200 and self.nuevo_tweet[2] <= 400  ):
            self.pond_final +=20
            self.listado_temporal.append(self.nuevo_tweet[2])  #Agregando cantidad de seguidores
            self.analisisAmigos()
        elif (self.nuevo_tweet[2] < 400 and self.nuevo_tweet[2] <= 1000  ):
            self.pond_final +=30
            self.listado_temporal.append(self.nuevo_tweet[2])  #Agregando cantidad de seguidores
            self.analisisAmigos()
        elif (self.nuevo_tweet[2] > 1000 ):
            self.pond_final +=35
            self.listado_temporal.append(self.nuevo_tweet[2])  #Agregando cantidad de seguidores
            self.analisisAmigos()        
        
    def analisisAmigos(self):               #Analisis de si la persona que escribio el tweet tiene muchos amigos que pueda retwittear
        if (self.nuevo_tweet[3] <= 50):
            self.pond_final +=0
            self.listado_temporal.append(self.nuevo_tweet[3])  #Agregando cantidad de amigos
            self.analisisViralizacionReply()
        elif (self.nuevo_tweet[3] < 50 and self.nuevo_tweet[3] <= 200  ):
            self.pond_final +=10
            self.listado_temporal.append(self.nuevo_tweet[3])  #Agregando cantidad de amigos
            self.analisisViralizacionReply()
        elif (self.nuevo_tweet[3] < 200 and self.nuevo_tweet[3] <= 400  ):
            self.pond_final +=20
            self.listado_temporal.append(self.nuevo_tweet[3])  #Agregando cantidad de amigos
            self.analisisViralizacionReply()
        elif (self.nuevo_tweet[3] < 400 and self.nuevo_tweet[3] <= 1000  ):
            self.pond_final +=30
            self.listado_temporal.append(self.nuevo_tweet[3])  #Agregando cantidad de amigos
            self.analisisViralizacionReply()
        elif (self.nuevo_tweet[3] > 1000 ):
            self.pond_final +=35
            self.analisisViralizacionReply()  
            
    def analisisViralizacionReply(self):         #Analisis de si el tweet escrito ha sido respondido y en que medida
        if (self.nuevo_tweet[4] <= 5):
            self.pond_final +=0
            self.listado_temporal.append(self.nuevo_tweet[4])  #Agregando veces reply
            self.analisisViralizacionRetwt()
            
        elif (self.nuevo_tweet[4] > 5 and self.nuevo_tweet[4] <= 20  ):
            self.pond_final +=2.5
            self.listado_temporal.append(self.nuevo_tweet[4])  #Agregando veces reply
            self.analisisViralizacionRetwt()
        elif (self.nuevo_tweet[4] > 20 and self.nuevo_tweet[4] <= 40  ):
            self.pond_final +=5
            self.listado_temporal.append(self.nuevo_tweet[4])  #Agregando veces reply
            self.analisisViralizacionRetwt()
        elif (self.nuevo_tweet[4] > 40 and self.nuevo_tweet[4] <= 100  ):
            self.pond_final +=7.5
            self.listado_temporal.append(self.nuevo_tweet[4])  #Agregando veces reply
            self.analisisViralizacionRetwt()
        elif (self.nuevo_tweet[4] > 100 ):
            self.pond_final +=10
            self.listado_temporal.append(self.nuevo_tweet[4])  #Agregando veces reply
            self.analisisViralizacionRetwt()   
            
            
    def analisisViralizacionRetwt(self):         #Analisis de si el tweet escrito ha sido respondido y en que medida
        if (self.nuevo_tweet[5] <= 5):
            self.pond_final +=0
            self.listado_temporal.append(self.nuevo_tweet[5])  #Agregando veces Retweet
            self.almacenamientoFinal()
        elif (self.nuevo_tweet[5] < 5 and self.nuevo_tweet[5] <= 20  ):
            self.pond_final +=2,5
            self.listado_temporal.append(self.nuevo_tweet[5])  #Agregando veces Retweet
            self.almacenamientoFinal()
        elif (self.nuevo_tweet[5] < 20 and self.nuevo_tweet[5] <= 40  ):
            self.pond_final +=5
            self.listado_temporal.append(self.nuevo_tweet[5])  #Agregando veces Retweet
            self.almacenamientoFinal()
        elif (self.nuevo_tweet[5] < 40 and self.nuevo_tweet[5] <= 100  ):
            self.pond_final +=7,5
            self.listado_temporal.append(self.nuevo_tweet[5])  #Agregando veces Retweet
            self.almacenamientoFinal()
        elif (self.nuevo_tweet[5] > 100 ):
            self.pond_final +=10
            self.listado_temporal.append(self.nuevo_tweet[5])  #Agregando veces Retweet
            self.almacenamientoFinal()         
    
    def almacenamientoFinal(self):
        self.listado_temporal.append(self.pond_final)  #Agregando ponderacion final
        self.listado_tweets_interes.append(self.listado_temporal)
        
        #df = pd.DataFrame(data= [self.listado_temporal], columns=['Texto_Tweet','Nombre_Usuario','Match_Tweet','Ubicacion_Usuario',
        #                                                        'Analisis_Sentimiento','Analisis_Num_Seguidores',
        #                                                        'Analisis_Num_Amigos','Analisis_Viral_Reply',
        #                                                        'Analisis_Viral_Retweet','Ponderacion_Tweet'])  #We're creating in pandas
        #df
        print('')
        print('El proceso ha llegado a su fin, el resultado a guardar en DataFrame es:')
        print('==================================================================================================')
        print(self.listado_temporal)
        print('==================================================================================================')
        self.listado_temporal = []
        
#Paso 1. Instanciar Activando Conexion
activation_key = ActivandoConexion(ACCESS_TOKEN, ACCESS_TOKEN_SECRET, API_KEY, API_KEY_SECRET)
auth_key = activation_key.GenerarAuthorization()

#Paso 2.Activando Tweepy
api = tweepy.API(auth_key, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#Paso 3.Instancia para activar Listener en Twitter
stream = TweetsListener()
streamingApi = tweepy.Stream(auth=api.auth, listener=stream)
streamingApi.filter(track=["#SOSGEB", "SOSGEB", "torres de energia", "falla energia", 
                           "fallas electricas", "torres de transmision", "robo de energia", 
                           "Grupo Energía Bogotá", "torres de energía", "falla energía", 
                           "fallas eléctricas", "avalancha en torres", "torres de transmisión", 
                           "atentado torres de transmisión", "robo de energía", "Ironhack"])#location=[-77.6556101233,-4.0252519154,-67.086762467,11.9029453551] )



Online

Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: #SOSGEB reportando inestabilidad torre energia transmision ineptos y corruptos v5 not great at all corrupts
La descomposicion del tweet es:
Fecha_Creacion 2021-09-17 23:11:18
Tweet ya impreso
followers_count 0
friends_count 0
reply_count 0
retweet_count 0
geo None
place None
location None
name Cristhian Ordonez

El proceso ha llegado a su fin, el resultado a guardar en DataFrame es:
['#SOSGEB reportando inestabilidad torre energia transmision ineptos y corruptos v5 not great at all corrupts', 'Cristhian Ordonez', 4, 'Sin Ubicacion', -0.4, 0, 0, 0, 0, 10]

Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: RT @maduradascom: ¡LO MÁS RECIENTE! Maduro responsabilizó a «terroristas» del incendio en subestación Santa Cruz que causó fallas eléctrica…
La descomposicion del tweet es:
Fecha_Creacion 2021-09-17 23:12:52
Tweet ya impreso
followers_count 2557
friends_count 1420
reply_count 0
retweet


Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: RT @CarlosEA17: @clacpi @JovenIndigena @ibermedia @recam_mercosur @ONIC_Colombia @CONAIE_Ecuador @FICWALLMAPU @lunamaran @agendapropiaap @m…
La descomposicion del tweet es:
Fecha_Creacion 2021-09-17 23:20:58
Tweet ya impreso
followers_count 194
friends_count 426
reply_count 0
retweet_count 0
geo None
place None
location None
name peter cifuentes
El Tweet que llego se desecho por no cumplir con los parametros

Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: RT @Minas_Energia: Incêndios fora de controle podem chegar a torres de transmissão e impedir o transporte de energia elétrica, deixando mil…
La descomposicion del tweet es:
Fecha_Creacion 2021-09-17 23:21:27
Tweet ya impreso
followers_count 2465
friends_count 3697
reply_count 0
retweet_count 0
geo None
place None
location None
name Origh Internacional

El proceso ha llegado a su fin, el resultado a guardar en DataFrame es:
['RT @Minas_Ene


Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: Falla en suministro de energía eléctrica afectó el sistema de ventilación en una avícola de Panamá Oeste, provocand… https://t.co/9DqTi9Uk5v
La descomposicion del tweet es:
Fecha_Creacion 2021-09-17 23:46:49
Tweet ya impreso
followers_count 719882
friends_count 1541
reply_count 0
retweet_count 0
geo None
place None
location #Panamá
name Telemetro Reporta

El proceso ha llegado a su fin, el resultado a guardar en DataFrame es:
['Falla en suministro de energía eléctrica afectó el sistema de ventilación en una avícola de Panamá Oeste, provocand… https://t.co/9DqTi9Uk5v', 'Telemetro Reporta', 1, '#Panamá', 0.0, 719882, 0, 0, 95]

Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: RT @TReporta: Falla en suministro de energía eléctrica afectó el sistema de ventilación en una avícola de Panamá Oeste, provocando la pérdi…
La descomposicion del tweet es:
Fecha_Creacion 2021-09-17 23:47:04
Tweet ya impr


Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: @CFE_Contigo Buenas tardes 
Quiero levantar un reporte de falla en el suministro de energia en mi domicilio
La descomposicion del tweet es:
Fecha_Creacion 2021-09-17 23:55:30
Tweet ya impreso
followers_count 0
friends_count 1
reply_count 0
retweet_count 0
geo None
place None
location None
name Lucia Navarro

El proceso ha llegado a su fin, el resultado a guardar en DataFrame es:
['@CFE_Contigo Buenas tardes \nQuiero levantar un reporte de falla en el suministro de energia en mi domicilio', 'Lucia Navarro', 2, 'Sin Ubicacion', 0.0, 0, 1, 0, 0, 10]

Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: RT @PinedaRuizBog: Es mentira lo que dice @ClaudiaLopez en este video: al fusionar Grupo de Energía de Bogotá (GEB) y ENEL lo que se consol…
La descomposicion del tweet es:
Fecha_Creacion 2021-09-17 23:55:40
Tweet ya impreso
followers_count 1049
friends_count 4999
reply_count 0
retweet_count 0
geo No


Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: @carmena13008308 @yoimerkis @Osirisdleon Me informaron que había falla en el sistema que la energía eléctrica estab… https://t.co/hjEFQFZMdJ
La descomposicion del tweet es:
Fecha_Creacion 2021-09-18 00:23:04
Tweet ya impreso
followers_count 1217
friends_count 899
reply_count 0
retweet_count 0
geo None
place Place(_api=<tweepy.api.API object at 0x000001B5EE3772E0>, id='0093a5dd1242be45', url='https://api.twitter.com/1.1/geo/id/0093a5dd1242be45.json', place_type='city', name='Santiago', full_name='Santiago, Dominican Republic', country_code='DO', country='Dominican Republic', bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x000001B5EE3772E0>, type='Polygon', coordinates=[[[-70.898907, 19.272001], [-70.898907, 19.639734], [-70.60514, 19.639734], [-70.60514, 19.272001]]]), attributes={})
location None
name teresita espinal

Ha llegado un nuevo Tweet, iniciando analisis...
El Tweet que llego es: Mi diagnóstico